## Iterative Adv Training

### For iter in Range(20):
-       Step 1: Generate training adv examples using weights from last epoch to 1) overwrite shot_adv.png 2) create coco_perturb_train.json
-       Step 2: Generate testing adv examples using weights from last epoch 1) overwrite shot_adv.png 2) create coco_perturb_test.json
-       Step 3: Mix downsampled normal training data(D0) with perturbed training data(Di) (4:1) and create coco_adv_mix.json
-       Step 4: Evaluate training Acc on all D0+Di examples (9 metrics)
-       Step 5: Evaluate testing Acc on clean testing 
-       Step 6: Evaluate testing Acc on perturbed testing (perturbed with weights from last epoch)

In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import pickle

In [2]:
import os
os.chdir('..')

In [3]:
!pwd

/home/l/liny/yizhe/code/detectron2-1


In [4]:
def evaluate_res(gt_coco_path, results_coco_path, output_this_epoch, name):
    coco_gt = COCO(gt_coco_path)
    coco_dt = coco_gt.loadRes(results_coco_path)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    with open(output_this_epoch + '/all_%sacc.pkl'%name, 'wb') as f:
        pickle.dump(coco_eval, f)
    
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.catIds = [1]
    with open(output_this_epoch + '/input_%sacc.pkl'%name, 'wb') as f:
        pickle.dump(coco_eval, f)
        
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.catIds = [2]
    with open(output_this_epoch + '/logo_%sacc.pkl'%name, 'wb') as f:
        pickle.dump(coco_eval, f)
        

In [ ]:

for i in range(5, 10):
    if i == 0:
        weight_last_epoch = 'output/rcnn_bet365/model_final.pth'
    else:
        weight_last_epoch = 'output/adv_%s/model_final.pth'%(str(i - 1))
    output_this_epoch = 'output/adv_%s'%(str(i))
    weight_this_epoch = output_this_epoch + '/model_final.pth'
    
    if not (os.path.exists(weight_last_epoch)):
        print('Weight for last epoch does not exist!')
        
    os.makedirs(output_this_epoch, exist_ok=True)
        
    '''generate training, testing adv examples'''
    !python run_DAG.py --test False --cfg-path configs/faster_rcnn_bet365.yaml \
                                               --weights-path {weight_last_epoch} \
                                               --results-save-path coco_instances_ignore_train.json 

    !python run_DAG.py --test True --cfg-path configs/faster_rcnn_bet365.yaml \
                                              --weights-path {weight_last_epoch} \
                                              --results-save-path coco_instances_ignore_test.json 
        
    #mix normal with training
    !python detectron2_1/mix_normal_adv.py --normal-path data/benign_data/coco_train.json \
                                           --adv-path data/benign_data/coco_perturbgt_train.json \
                                           --save-path data/benign_data/coco_adv_mix.json 

    #retrain 
    !python train_net.py --config-file configs/faster_rcnn_adv.yaml \
                                       OUTPUT_DIR {output_this_epoch} \
                                       MODEL.WEIGHTS {weight_last_epoch} 
    
    ######################## evaluate on training D0 + Di ################################################ 
    !python train_net.py --eval-only --config-file configs/faster_rcnn_adv_eval_train.yaml \
                                                   MODEL.WEIGHTS {weight_this_epoch} \
                                                   OUTPUT_DIR {output_this_epoch} # Path to trained checkpoint
    
    gt_coco_path = 'data/benign_data/coco_adv_mix.json' # here the ground-truth is for the original-sized image
    results_coco_path =  output_this_epoch + '/coco_instances_results.json' # here the prediction is for the resized image
    evaluate_res(gt_coco_path, results_coco_path, output_this_epoch, 'train')
        
    #########################  evaluate on clean testing ################################################ 
    !python train_net.py --eval-only --config-file configs/faster_rcnn_adv.yaml \
                                     MODEL.WEIGHTS {weight_this_epoch} \
                                     OUTPUT_DIR {output_this_epoch} # Path to trained checkpoint
    
    gt_coco_path = 'data/benign_data/coco_test.json' 
    results_coco_path =  output_this_epoch + '/coco_instances_results.json' 
    evaluate_res(gt_coco_path, results_coco_path, output_this_epoch, 'test')

        
    #########################  evaluate on perturbed testing ################################################ 
    !python train_net.py --eval-only --config-file configs/faster_rcnn_adv_eval_test.yaml \
                                     MODEL.WEIGHTS {weight_this_epoch} \
                                     OUTPUT_DIR {output_this_epoch} # Path to trained checkpoint
    
    gt_coco_path = 'data/benign_data/coco_perturbgt_test.json' 
    results_coco_path =  output_this_epoch + '/coco_instances_results.json' 
    evaluate_res(gt_coco_path, results_coco_path, output_this_epoch, 'perturb_test')



[3744/29073] Attacking data/benign_data/benign_database/infoq.com/shot.png ...
Done with attack. Total Iterations 149
3745it [12:28:40, 12.06s/it][3745/29073] Attacking data/benign_data/benign_database/liveberlin.ru/shot.png ...
Done with attack. Total Iterations 149
3746it [12:28:53, 12.33s/it][3746/29073] Attacking data/benign_data/benign_database/pdhpe.net/shot.png ...
Done with attack. Total Iterations 149
3747it [12:29:06, 12.57s/it][3747/29073] Attacking data/benign_data/benign_database/eventective.com/shot.png ...
Done with attack. Total Iterations 149
3748it [12:29:16, 12.01s/it][3748/29073] Attacking data/benign_data/benign_database/dasunerwartete.nl/shot.png ...
Done with attack. Total Iterations 149
3749it [12:29:29, 12.16s/it][3749/29073] Attacking data/benign_data/benign_database/abvshka.ru/shot.png ...
Done with attack. Total Iterations 149
3750it [12:29:38, 11.34s/it][3750/29073] Attacking data/benign_data/benign_database/longroom.com/shot.png ...
Done with attack. Total

Done with attack. Total Iterations 149
3799it [12:39:10, 11.51s/it][3799/29073] Attacking data/benign_data/benign_database/merolagani.com/shot.png ...
Done with attack. Total Iterations 149
3800it [12:39:21, 11.26s/it][3800/29073] Attacking data/benign_data/benign_database/themetrust.com/shot.png ...
Done with attack. Total Iterations 149
3801it [12:39:32, 11.27s/it][3801/29073] Attacking data/benign_data/benign_database/eiseverywhere.com/shot.png ...
Done with attack. Total Iterations 149
3802it [12:39:45, 11.64s/it][3802/29073] Attacking data/benign_data/benign_database/nuevosvecinos.com/shot.png ...
Done with attack. Total Iterations 149
3803it [12:39:55, 11.31s/it][3803/29073] Attacking data/benign_data/benign_database/miniplay.com/shot.png ...
Done with attack. Total Iterations 149
3804it [12:40:08, 11.81s/it][3804/29073] Attacking data/benign_data/benign_database/frankwatching.com/shot.png ...
Done with attack. Total Iterations 0
3805it [12:40:10,  8.64s/it][3805/29073] Attacking

Done with attack. Total Iterations 149
3854it [12:49:12,  9.33s/it][3854/29073] Attacking data/benign_data/benign_database/elizabethsuzann.com/shot.png ...
Done with attack. Total Iterations 149
3855it [12:49:24, 10.29s/it][3855/29073] Attacking data/benign_data/benign_database/tekstovoi.ru/shot.png ...
Done with attack. Total Iterations 149
3856it [12:49:37, 10.98s/it][3856/29073] Attacking data/benign_data/benign_database/britishcouncil.az/shot.png ...
Done with attack. Total Iterations 149
3857it [12:49:50, 11.58s/it][3857/29073] Attacking data/benign_data/benign_database/hisour.com/shot.png ...
Done with attack. Total Iterations 149
3858it [12:50:00, 11.31s/it][3858/29073] Attacking data/benign_data/benign_database/fastjobs.sg/shot.png ...
Done with attack. Total Iterations 149
3859it [12:50:09, 10.64s/it][3859/29073] Attacking data/benign_data/benign_database/mizuho-sc.com/shot.png ...
Done with attack. Total Iterations 149
3860it [12:50:21, 10.85s/it][3860/29073] Attacking data/b

Done with attack. Total Iterations 149
3909it [12:59:19, 11.30s/it][3909/29073] Attacking data/benign_data/benign_database/adeevee.com/shot.png ...
Done with attack. Total Iterations 149
3910it [12:59:27, 10.15s/it][3910/29073] Attacking data/benign_data/benign_database/delfontmackintosh.co.uk/shot.png ...
Done with attack. Total Iterations 149
3911it [12:59:40, 10.95s/it][3911/29073] Attacking data/benign_data/benign_database/instaling.pl/shot.png ...
Done with attack. Total Iterations 149
3912it [12:59:52, 11.46s/it][3912/29073] Attacking data/benign_data/benign_database/vchat.vn/shot.png ...
Done with attack. Total Iterations 149
3913it [13:00:05, 11.86s/it][3913/29073] Attacking data/benign_data/benign_database/mostphotos.com/shot.png ...
Done with attack. Total Iterations 149
3914it [13:00:15, 11.20s/it][3914/29073] Attacking data/benign_data/benign_database/theoldreader.com/shot.png ...
Done with attack. Total Iterations 149
3915it [13:00:26, 11.29s/it][3915/29073] Attacking data

Done with attack. Total Iterations 149
3964it [13:09:34, 10.88s/it][3964/29073] Attacking data/benign_data/benign_database/nette.org/shot.png ...
Done with attack. Total Iterations 149
3965it [13:09:45, 11.03s/it][3965/29073] Attacking data/benign_data/benign_database/picsee.co/shot.png ...
Done with attack. Total Iterations 149
3966it [13:09:58, 11.44s/it][3966/29073] Attacking data/benign_data/benign_database/phpfox.com/shot.png ...
Done with attack. Total Iterations 149
3967it [13:10:08, 11.18s/it][3967/29073] Attacking data/benign_data/benign_database/fairfx.com/shot.png ...
Done with attack. Total Iterations 149
3968it [13:10:18, 10.78s/it][3968/29073] Attacking data/benign_data/benign_database/vadiro.com/shot.png ...
Done with attack. Total Iterations 149
3969it [13:10:28, 10.50s/it][3969/29073] Attacking data/benign_data/benign_database/ipool.se/shot.png ...
Done with attack. Total Iterations 149
3970it [13:10:38, 10.50s/it][3970/29073] Attacking data/benign_data/benign_database

...

1188it [3:37:28, 11.04s/it][1188/29073] Attacking data/benign_data/benign_database/amz.one/shot.png ...
Done with attack. Total Iterations 149
1189it [3:37:41, 11.45s/it][1189/29073] Attacking data/benign_data/benign_database/zopso.com/shot.png ...
Done with attack. Total Iterations 149
1190it [3:37:50, 10.97s/it][1190/29073] Attacking data/benign_data/benign_database/itoigawahisui.jp/shot.png ...
Done with attack. Total Iterations 149
1191it [3:38:02, 11.16s/it][1191/29073] Attacking data/benign_data/benign_database/microbiologysociety.org/shot.png ...
Done with attack. Total Iterations 149
1192it [3:38:12, 10.82s/it][1192/29073] Attacking data/benign_data/benign_database/exoclick.com/shot.png ...
Done with attack. Total Iterations 149
1193it [3:38:26, 11.62s/it][1193/29073] Attacking data/benign_data/benign_database/maiotaku.com/shot.png ...
Done with attack. Total Iterations 149
1194it [3:38:38, 11.93s/it][1194/29073] Attacking data/benign_data/benign_database/houzz.co.uk/shot.png .

1243it [3:47:31, 11.27s/it][1243/29073] Attacking data/benign_data/benign_database/layoutit.com/shot.png ...
Done with attack. Total Iterations 149
1244it [3:47:43, 11.63s/it][1244/29073] Attacking data/benign_data/benign_database/xpressdocs.com/shot.png ...
Done with attack. Total Iterations 149
1245it [3:47:52, 10.91s/it][1245/29073] Attacking data/benign_data/benign_database/canadaluggagedepot.ca/shot.png ...
Done with attack. Total Iterations 149
1246it [3:48:02, 10.52s/it][1246/29073] Attacking data/benign_data/benign_database/fxsolver.com/shot.png ...
Done with attack. Total Iterations 149
1247it [3:48:14, 11.05s/it][1247/29073] Attacking data/benign_data/benign_database/nevosoft.ru/shot.png ...
Done with attack. Total Iterations 149
1248it [3:48:26, 11.23s/it][1248/29073] Attacking data/benign_data/benign_database/bakuelectronics.az/shot.png ...
Done with attack. Total Iterations 149
1249it [3:48:38, 11.49s/it][1249/29073] Attacking data/benign_data/benign_database/tanea.gr/shot

1298it [3:57:58, 11.01s/it][1298/29073] Attacking data/benign_data/benign_database/cheng-tsui.com/shot.png ...
Done with attack. Total Iterations 149
1299it [3:58:11, 11.42s/it][1299/29073] Attacking data/benign_data/benign_database/lyrikline.org/shot.png ...
Done with attack. Total Iterations 149
1300it [3:58:22, 11.53s/it][1300/29073] Attacking data/benign_data/benign_database/iesa.org/shot.png ...
Done with attack. Total Iterations 149
1301it [3:58:35, 11.78s/it][1301/29073] Attacking data/benign_data/benign_database/yesrewardz.com/shot.png ...
Done with attack. Total Iterations 149
1302it [3:58:47, 11.91s/it][1302/29073] Attacking data/benign_data/benign_database/zddream.com/shot.png ...
Done with attack. Total Iterations 149
1303it [3:59:00, 12.11s/it][1303/29073] Attacking data/benign_data/benign_database/lumigreen.sk/shot.png ...
Done with attack. Total Iterations 149
1304it [3:59:09, 11.17s/it][1304/29073] Attacking data/benign_data/benign_database/switchbrew.org/shot.png ...
D

1353it [4:08:28,  9.83s/it][1353/29073] Attacking data/benign_data/benign_database/alcione.biz/shot.png ...
Done with attack. Total Iterations 149
1354it [4:08:41, 10.60s/it][1354/29073] Attacking data/benign_data/benign_database/open.ru/shot.png ...
Done with attack. Total Iterations 149
1355it [4:08:50, 10.36s/it][1355/29073] Attacking data/benign_data/benign_database/indiasfreeclassified.com/shot.png ...
Done with attack. Total Iterations 149
1356it [4:09:00, 10.22s/it][1356/29073] Attacking data/benign_data/benign_database/creagames.com/shot.png ...
Done with attack. Total Iterations 149
1357it [4:09:13, 10.95s/it][1357/29073] Attacking data/benign_data/benign_database/hsrpgujarat.com/shot.png ...
Done with attack. Total Iterations 149
1358it [4:09:24, 11.05s/it][1358/29073] Attacking data/benign_data/benign_database/logicsupply.com/shot.png ...
Done with attack. Total Iterations 149
1359it [4:09:32, 10.03s/it][1359/29073] Attacking data/benign_data/benign_database/insightexpressai

Done with attack. Total Iterations 149
1409it [4:18:41, 10.90s/it][1409/29073] Attacking data/benign_data/benign_database/cricketworldcup.com/shot.png ...
Done with attack. Total Iterations 149
1410it [4:18:53, 11.41s/it][1410/29073] Attacking data/benign_data/benign_database/evernote.com/shot.png ...
Done with attack. Total Iterations 149
1411it [4:19:04, 11.33s/it][1411/29073] Attacking data/benign_data/benign_database/zygorguides.com/shot.png ...
Done with attack. Total Iterations 149
1412it [4:19:17, 11.57s/it][1412/29073] Attacking data/benign_data/benign_database/uniz.com/shot.png ...
Done with attack. Total Iterations 149
1413it [4:19:26, 10.97s/it][1413/29073] Attacking data/benign_data/benign_database/mytrendyphone.no/shot.png ...
Done with attack. Total Iterations 149
1414it [4:19:37, 11.07s/it][1414/29073] Attacking data/benign_data/benign_database/redacteur.com/shot.png ...
Done with attack. Total Iterations 149
1415it [4:19:51, 11.69s/it][1415/29073] Attacking data/benign_

In [ ]:
with open('./output/adv_1/logo_testacc.pkl', 'rb') as f:
    logo_testacc = pickle.load(f)
    
with open('./output/adv_1/input_testacc.pkl', 'rb') as f:
    input_testacc = pickle.load(f)
    
with open('./output/adv_1/all_testacc.pkl', 'rb') as f:
    all_testacc = pickle.load(f)

In [ ]:
with open('./output/adv_1/logo_trainacc.pkl', 'rb') as f:
    logo_trainacc = pickle.load(f)
    
with open('./output/adv_1/input_trainacc.pkl', 'rb') as f:
    input_trainacc = pickle.load(f)
    
with open('./output/adv_1/all_trainacc.pkl', 'rb') as f:
    all_trainacc = pickle.load(f)

In [ ]:
logo_testacc.evaluate()
logo_testacc.accumulate()
logo_testacc.summarize()